In [1]:
import numpy as np
import pandas as pd
import geocoder
import json 
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
import folium
print('Downloaded Library')

Downloaded Library


In [2]:
pwd

'C:\\Users\\jaklo\\Data_Docs'

In [3]:
lagos = pd.read_csv('C://Users//jaklo//Data_Docs/Lagos Neighbourhoods by Fallingrain.csv')
lagos.head()

,Name,What,Region,Country,Lat,Long,Elev ft.,Pop est
0,Aba,city,Lagos,Nigeria,6.49,3.38,118,561845
1,Aba-Onigbagbo,city,Lagos,Nigeria,6.5,4.09,137,3171
2,Aba-Oriyanrin,city,Lagos,Nigeria,6.45,4.14,157,4473
3,Aba-Oriyanrn,city,Lagos,Nigeria,6.45,4.14,157,4473
4,Abagbo,city,Lagos,Nigeria,6.4,3.38,114,179962


In [4]:
lagos = lagos[['Name', 'Region', 'Lat', 'Long', 'Pop est']]
lagos.head()

,Name,Region,Lat,Long,Pop est
0,Aba,Lagos,6.49,3.38,561845
1,Aba-Onigbagbo,Lagos,6.5,4.09,3171
2,Aba-Oriyanrin,Lagos,6.45,4.14,4473
3,Aba-Oriyanrn,Lagos,6.45,4.14,4473
4,Abagbo,Lagos,6.4,3.38,179962


In [5]:
lagos.shape

(565, 5)

In [6]:
lagos.dtypes

Name       object
Region     object
Lat        object
Long       object
Pop est    object
dtype: object

In [7]:
lagos.rename(columns = {'Pop est': 'Pop_est', 'Lat':'Latitude', 'Long':'Longitude'}, inplace=True)
print(lagos.isnull().sum())

Name         0
Region       0
Latitude     0
Longitude    0
Pop_est      0
dtype: int64


In [8]:
pd.to_numeric(lagos['Pop_est'], errors='coerce').notnull().all()

False

In [9]:
boolens = lagos['Pop_est']
boolens = pd.Series(boolens)
is_long = boolens.str.isnumeric()
lagos[is_long].head()

,Name,Region,Latitude,Longitude,Pop_est
0,Aba,Lagos,6.49,3.38,561845
1,Aba-Onigbagbo,Lagos,6.5,4.09,3171
2,Aba-Oriyanrin,Lagos,6.45,4.14,4473
3,Aba-Oriyanrn,Lagos,6.45,4.14,4473
4,Abagbo,Lagos,6.4,3.38,179962


In [10]:
lagos = lagos[is_long]
lagos['Pop_est'] = lagos.Pop_est.astype(int)
lagos['Latitude'] = lagos.Latitude.astype(float)
lagos['Longitude'] = lagos.Longitude.astype(float)
lagos.dtypes

Name          object
Region        object
Latitude     float64
Longitude    float64
Pop_est        int32
dtype: object

In [11]:
lagos_2 = lagos[lagos.Pop_est >= 800000]
print(lagos_2.shape)
lagos_2.head()

(126, 5)


,Name,Region,Latitude,Longitude,Pop_est
13,Abule Ijesha,Lagos,6.52,3.39,932797
15,Abule-Egba,Lagos,6.64,3.29,894666
17,Abule-Okuta,Lagos,6.54,3.40,1039676
20,Adekunle,Lagos,6.61,3.36,1364004
33,Agboyi,Lagos,6.58,3.41,1002001


In [12]:
lagos_2.reset_index(inplace=True)
lagos_2 = lagos_2[['Name', 'Latitude', 'Longitude', 'Pop_est']]
lagos_2.head()

,Name,Latitude,Longitude,Pop_est
0,Abule Ijesha,6.52,3.39,932797
1,Abule-Egba,6.64,3.29,894666
2,Abule-Okuta,6.54,3.40,1039676
3,Adekunle,6.61,3.36,1364004
4,Agboyi,6.58,3.41,1002001


In [13]:
lagos_2.rename(columns = {'Name':'Neighbourhoods'}, inplace=True)
lagos_2.head()

,Neighbourhoods,Latitude,Longitude,Pop_est
0,Abule Ijesha,6.52,3.39,932797
1,Abule-Egba,6.64,3.29,894666
2,Abule-Okuta,6.54,3.40,1039676
3,Adekunle,6.61,3.36,1364004
4,Agboyi,6.58,3.41,1002001


## Explore and Cluster Neibourhoods in Lagos 

### Using Foursquare API to explore Lagos

In [14]:
CLIENT_ID = '3KH1DV2LCDYXWX1PWZWP1VYVJWFKRHIXICVWDOR0XY0CFBHI'
CLIENT_SECRET = 'TQWI25K0B04X5W1LPFQ5FGJYH32HIOKSWGD41PGL13ERAQRW'
VERSION = '20190605'

In [15]:
neighbourhood_latitude = lagos_2.iloc[0,1]
neighbourhood_longitude = lagos_2.iloc[0,2]
neighbourhood_name = lagos_2.iloc[0,0]
print('The Latitude and Longitude values of {}, are {},{}.'.format(neighbourhood_name,
                                                                  neighbourhood_latitude,
                                                                  neighbourhood_longitude))

The Latitude and Longitude values of Abule Ijesha, are 6.52,3.39.


In [16]:
LIMIT = 100
radius = 750
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighbourhood_latitude,
    neighbourhood_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=3KH1DV2LCDYXWX1PWZWP1VYVJWFKRHIXICVWDOR0XY0CFBHI&client_secret=TQWI25K0B04X5W1LPFQ5FGJYH32HIOKSWGD41PGL13ERAQRW&v=20190605&ll=6.52,3.39&radius=750&limit=100'

In [17]:
#Get requests and examine results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0ed279da5161773b3e288e'},
 'response': {'headerLocation': 'Yaba',
  'headerFullLocation': 'Yaba, Lagos',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 6.526750006750007,
    'lng': 3.396781263027096},
   'sw': {'lat': 6.5132499932499925, 'lng': 3.3832187369729043}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d0c7cd93bbaa143de6f36da',
       'name': 'Yem Yem Supermarket',
       'location': {'address': 'University Road',
        'lat': 6.518628850157461,
        'lng': 3.3921824191509895,
        'labeledLatLngs': [{'label': 'display',
          'lat': 6.518628850157461,
          'lng': 3.3921824191509895}],
        'distance': 285,
        'cc': 'NG',
        'city': '

In [18]:
#Borrow get_category_type from Foursquare
def get_category_type(row):
    try:
        categories_list = row['categories']
    except: 
        categories_list = row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
#Clean json and structure ot into a pandas DataFrame 
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

#Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

#Filter category for each row
nearby_venues['venues.categories'] = nearby_venues.apply(get_category_type, axis=1)

#Clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,count,items,referralId,categories,id,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,state,name,count,groups,categories
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d0c7cd93bbaa143de6f36da-0,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",4d0c7cd93bbaa143de6f36da,University Road,NG,Lagos,Nigeria,NaN,285,"[University Road, Lagos, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.518628850157461...",6.518629,3.392182,Lagos,Yem Yem Supermarket,0,[],Department Store
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cb1c2b6cbab236a1441ab73-1,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",4cb1c2b6cbab236a1441ab73,Unilag,NG,Lagos,Nigeria,NaN,184,"[Unilag, Lagos, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.519301759009386...",6.519302,3.391513,Lagos,Cafeteria,0,[],Restaurant
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-50839f82e4b0e99d0c119350-2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",50839f82e4b0e99d0c119350,Unilag,NG,Lagos,Nigeria,Akoka,475,"[Unilag (Akoka), Lagos, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.517772110734697...",6.517772,3.386328,Lagos,Unilag Car Park,0,[],Park
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ec804909a52756c5656c3b6-3,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",4ec804909a52756c5656c3b6,"University Road, Unilag",NG,Akoka,Nigeria,NaN,495,"[University Road, Unilag, Akoka, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.518648881071559...",6.518649,3.394266,Lagos,Unilag Community Pharmacy,0,[],Pharmacy
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4fa0dd34e4b0cca383f5af08-4,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",4fa0dd34e4b0cca383f5af08,NaN,NG,Yaba,Nigeria,Akoka,707,"[Akoka, Yaba, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.524552299124984...",6.524552,3.385535,Lagos,Pako Bus-Stop,0,[],Bus Station


In [20]:
#How many Venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [21]:
#Create Funtion to iterate the process with all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng in zip (names, latitudes, longitudes):
        print(name)
        
        #Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #Return only relevant info for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return (nearby_venues) 

In [22]:
Lg_venues = getNearbyVenues(names = lagos_2['Neighbourhoods'],
                           latitudes = lagos_2['Latitude'],
                           longitudes = lagos_2['Longitude']) 

Abule Ijesha
Abule-Egba
Abule-Okuta
Adekunle
Agboyi
Agege
Agidingbi
Agodo
Aguda
Ajegunle
Akangba
Akoka
Akowonjo
Alagba
Alimosho
Alimoso
Anthony
Awja-Ikoradu
Bariga
Beshi
Coker
Ebute Ikorodu
Ebute-Egga
Egba
Egbe
Egbeda
Ejigbo
Ekoro
Eregun
Erukan
Erunkan
Erutan
Ewu
Fadeyi
Fagba
Gbagada
Gbesse
Gbogbo
George
Hausa
Ibese
Ibeshe
Ibeshi
Idi Oro
Idi-Araba
Idioro
Ifako
Ifako-Agege
Igbobi
Ijaiye
Iju
Iju Junction
Iju Water Works
Ikate
Ikeja
Ikeja G.R.A.
Ikeja Government Reserved Area
Ikosi-Ketu
Ilasamaja
Ilupeju
Isaga-Tedo
Isagatedo
Isasi
Ishaga
Isheri Olofin
Isheri-Olofin
Isolo
Jibowu
Ketu
Libowu
Mafoluku
Magbon
Magodo
Magodo I
Magodo II
Maryland
Mende
Mile 12
Mile Twelve
Mofoluku
Mushin
Mushin Halt
New Lagos
Obanikoro
Oguau
Ogudu
Ojodu
Ojota
Ojudo
Oke Afa
Okunola
Olowoira
Olowora
Olusosun
Omole
Onigbongbo
Onikosi
Onipanu
Onisigun
Opamore
Opebi
Oregun
Orile Oshodi
Orishigun
Oruba
Oruba Agboyi
Oshodi
Oshosun
Ososun
Ososun
Oworonshoki
Oworonsoki
Papa Ajao
Papa-Ashafa
Pedro
Sagisa
Shangisha
Shogunl

In [23]:
print(Lg_venues.shape)
Lg_venues.head()

(357, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abule Ijesha,6.52,3.39,Yem Yem Supermarket,6.518629,3.392182,Department Store
1,Abule Ijesha,6.52,3.39,Cafeteria,6.519302,3.391513,Restaurant
2,Abule Ijesha,6.52,3.39,Unilag Car Park,6.517772,3.386328,Park
3,Abule Ijesha,6.52,3.39,Unilag Community Pharmacy,6.518649,3.394266,Pharmacy
4,Abule-Egba,6.64,3.29,Agbelekale,6.641512,3.287590,Bus Stop


In [24]:
Lg_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Abule Ijesha,4,4,4,4,4,4
Abule-Egba,1,1,1,1,1,1
Abule-Okuta,2,2,2,2,2,2
Adekunle,2,2,2,2,2,2
Agege,1,1,1,1,1,1
Agidingbi,6,6,6,6,6,6
Agodo,2,2,2,2,2,2
Aguda,4,4,4,4,4,4
Ajegunle,4,4,4,4,4,4


In [25]:
#How many unique categories can be curated from returned venues
print('There are {} unique categories.'.format(len(Lg_venues['Venue Category'].unique())))

There are 83 unique categories.


In [26]:
#Analysis of each Neighbourhood
lg_onehot = pd.get_dummies(Lg_venues[['Venue Category']], prefix = "", prefix_sep="")
lg_onehot['Neighbourhood'] = Lg_venues['Neighbourhood']
#Move column to first column
fixed_columns = [lg_onehot.columns[-1]]+list(lg_onehot.columns[:-1])
lg_onehot.head()

,Accessories Store,African Restaurant,Airport,Airport Terminal,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,...,Snack Place,Soup Place,Spa,Stadium,Steakhouse,Supermarket,Tailor Shop,Trail,Wine Bar,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Abule Ijesha
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Abule Ijesha
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Abule Ijesha
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Abule Ijesha
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Abule-Egba


In [27]:
#Group rows by neighbourhood and by taking mean of the frequency of occurence of each category
lg_grouped = lg_onehot.groupby('Neighbourhood').mean().reset_index()
lg_grouped.head()

,Neighbourhood,Accessories Store,African Restaurant,Airport,Airport Terminal,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Auto Garage,...,Shopping Mall,Snack Place,Soup Place,Spa,Stadium,Steakhouse,Supermarket,Tailor Shop,Trail,Wine Bar
0,Abule Ijesha,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abule-Egba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abule-Okuta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Adekunle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Agege,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#Write function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

#Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#Create new DataFrame
neighbourhoods_venues_sorted = pd.DataFrame(columns = columns)
neighbourhoods_venues_sorted['Neighbourhood'] = lg_grouped['Neighbourhood']

for ind in np.arange(lg_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(lg_grouped.iloc[ind,:], num_top_venues)
    
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abule Ijesha,Restaurant,Park,Pharmacy,Department Store,Cupcake Shop
1,Abule-Egba,Bus Stop,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
2,Abule-Okuta,Bus Station,Hotel,Wine Bar,Diner,Clothing Store
3,Adekunle,Food Court,Coffee Shop,Wine Bar,Department Store,Chinese Restaurant
4,Agege,Bar,Wine Bar,Department Store,Chinese Restaurant,Clothing Store


### Cluster Neighbourhoods using k-means

In [30]:
#set initial number of clusters
kclusters = 8

lg_grouped_clustering = lg_grouped.drop('Neighbourhood', 1)

#run k-means clustering
kmeans = KMeans (n_clusters = kclusters, random_state=0).fit(lg_grouped_clustering)

#Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 2, 0, 3, 0, 0, 0, 0, 0])

In [33]:
#add clusters labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
lagos_2.rename(columns={'Neighbourhoods':'Neighbourhood'}, inplace=True)
lg_merged = lagos_2
lg_merged = lg_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
lg_merged.head()

,Neighbourhood,Latitude,Longitude,Pop_est,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abule Ijesha,6.52,3.39,932797,0.0,Restaurant,Park,Pharmacy,Department Store,Cupcake Shop
1,Abule-Egba,6.64,3.29,894666,1.0,Bus Stop,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
2,Abule-Okuta,6.54,3.40,1039676,2.0,Bus Station,Hotel,Wine Bar,Diner,Clothing Store
3,Adekunle,6.61,3.36,1364004,0.0,Food Court,Coffee Shop,Wine Bar,Department Store,Chinese Restaurant
4,Agboyi,6.58,3.41,1002001,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
print(lg_merged.isnull().sum())
lg_merged.rename(columns={'Cluster Labels':'Cluster_Labels'}, inplace=True)
lg_merged[lg_merged.Cluster_Labels.isnull()]

Neighbourhood             0
Latitude                  0
Longitude                 0
Pop_est                   0
Cluster Labels           24
1st Most Common Venue    24
2nd Most Common Venue    24
3rd Most Common Venue    24
4th Most Common Venue    24
5th Most Common Venue    24
dtype: int64


,Neighbourhood,Latitude,Longitude,Pop_est,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Agboyi,6.58,3.41,1002001,NaN,NaN,NaN,NaN,NaN,NaN
19,Beshi,6.56,3.47,924701,NaN,NaN,NaN,NaN,NaN,NaN
23,Egba,6.65,3.30,954844,NaN,NaN,NaN,NaN,NaN,NaN
25,Egbeda,6.59,3.30,1102565,NaN,NaN,NaN,NaN,NaN,NaN
26,Ejigbo,6.55,3.31,1313648,NaN,NaN,NaN,NaN,NaN,NaN
27,Ekoro,6.65,3.29,802037,NaN,NaN,NaN,NaN,NaN,NaN
34,Fagba,6.67,3.32,882354,NaN,NaN,NaN,NaN,NaN,NaN
36,Gbesse,6.56,3.47,924701,NaN,NaN,NaN,NaN,NaN,NaN
37,Gbogbo,6.59,3.52,838036,NaN,NaN,NaN,NaN,NaN,NaN
40,Ibese,6.56,3.47,924701,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
print(lg_merged.shape)

(126, 10)


In [36]:
lg_merged.dropna(inplace=True)
print(lg_merged.shape)
lg_merged.head()

(102, 10)


,Neighbourhood,Latitude,Longitude,Pop_est,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abule Ijesha,6.52,3.39,932797,0.0,Restaurant,Park,Pharmacy,Department Store,Cupcake Shop
1,Abule-Egba,6.64,3.29,894666,1.0,Bus Stop,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
2,Abule-Okuta,6.54,3.40,1039676,2.0,Bus Station,Hotel,Wine Bar,Diner,Clothing Store
3,Adekunle,6.61,3.36,1364004,0.0,Food Court,Coffee Shop,Wine Bar,Department Store,Chinese Restaurant
5,Agege,6.62,3.33,1326371,3.0,Bar,Wine Bar,Department Store,Chinese Restaurant,Clothing Store


In [37]:
from geopy.exc import GeocoderTimedOut
from time import sleep
address = 'Lagos, NG'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
sleep(1)

print('The geographical coordinates of Lagos are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Lagos are 6.4550575, 3.3941795.


In [38]:
#Visualize resulting clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [39]:
#add markers to the map
markers_colors = []

for lat,lon, poi, cluster in zip(lg_merged['Latitude'], lg_merged['Longitude'], lg_merged['Neighbourhood'], lg_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [40]:
#Cluster 1
lg_merged.loc[lg_merged['Cluster_Labels']==0, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abule Ijesha,Restaurant,Park,Pharmacy,Department Store,Cupcake Shop
3,Adekunle,Food Court,Coffee Shop,Wine Bar,Department Store,Chinese Restaurant
6,Agidingbi,Hotel,Music Venue,Park,Performing Arts Venue,Auto Dealership
7,Agodo,Newsstand,Flower Shop,Wine Bar,Department Store,Chinese Restaurant
8,Aguda,Candy Store,Music Venue,Fast Food Restaurant,Ice Cream Shop,Cupcake Shop
9,Ajegunle,Bus Station,African Restaurant,Park,Pharmacy,Cupcake Shop
10,Akangba,Paper / Office Supplies Store,Pizza Place,Fast Food Restaurant,Cupcake Shop,Candy Store
11,Akoka,Boat or Ferry,Wine Bar,Department Store,Chinese Restaurant,Clothing Store
12,Akowonjo,Bank,Bus Station,Airport Terminal,Market,Wine Bar
16,Anthony,Bus Station,Hotel,Shopping Mall,Park,Fast Food Restaurant


In [43]:
#Cluster 2
lg_merged.loc[lg_merged['Cluster_Labels']==1, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Abule-Egba,Bus Stop,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
13,Alagba,Bus Stop,African Restaurant,Diner,Chinese Restaurant,Clothing Store
66,Isolo,Bus Stop,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop


In [44]:
#Cluster 3
lg_merged.loc[lg_merged['Cluster_Labels']==2, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Abule-Okuta,Bus Station,Hotel,Wine Bar,Diner,Clothing Store
14,Alimosho,Hotel,Athletics & Sports,Wine Bar,Diner,Clothing Store
15,Alimoso,Hotel,Athletics & Sports,Wine Bar,Diner,Clothing Store
29,Erukan,Hotel,Wine Bar,Diner,Chinese Restaurant,Clothing Store
30,Erunkan,Hotel,Wine Bar,Diner,Chinese Restaurant,Clothing Store
31,Erutan,Hotel,Wine Bar,Diner,Chinese Restaurant,Clothing Store
55,Ikeja G.R.A.,Hotel,Wine Bar,Diner,Chinese Restaurant,Clothing Store
56,Ikeja Government Reserved Area,Hotel,Wine Bar,Diner,Chinese Restaurant,Clothing Store
71,Magbon,Business Service,Hotel,Diner,Chinese Restaurant,Clothing Store
93,Olusosun,Hotel,Wine Bar,Diner,Chinese Restaurant,Clothing Store


In [45]:
#Cluster 4
lg_merged.loc[lg_merged['Cluster_Labels']==3, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Agege,Bar,Wine Bar,Department Store,Chinese Restaurant,Clothing Store
51,Iju Junction,Bar,Wine Bar,Department Store,Chinese Restaurant,Clothing Store
58,Ilasamaja,Clothing Store,Bar,Wine Bar,Department Store,Chinese Restaurant
80,Mushin,Flea Market,Bar,Wine Bar,Department Store,Chinese Restaurant
81,Mushin Halt,Flea Market,Bar,Wine Bar,Department Store,Chinese Restaurant


In [46]:
#Cluster 5
lg_merged.loc[lg_merged['Cluster_Labels']==4, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
46,Ifako,Stadium,Wine Bar,Diner,Chinese Restaurant,Clothing Store
82,New Lagos,Stadium,African Restaurant,Bus Station,Convenience Store,Wine Bar
83,Obanikoro,African Restaurant,Wine Bar,Diner,Chinese Restaurant,Clothing Store
98,Onisigun,Convenience Store,Bus Station,Bank,Wine Bar,Department Store
103,Orishigun,Convenience Store,Bus Station,Bank,Wine Bar,Department Store
120,Sufu-Lere,Stadium,African Restaurant,Bus Station,Convenience Store,Wine Bar
121,Suru-Lere,Stadium,African Restaurant,Bus Station,Convenience Store,Wine Bar
122,Surulere,Stadium,African Restaurant,Bus Station,Convenience Store,Wine Bar


In [48]:
#Cluster 6
lg_merged.loc[lg_merged['Cluster_Labels']==5, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,Bariga,Bus Station,Wine Bar,Department Store,Chinese Restaurant,Clothing Store
67,Jibowu,Bus Station,Farmers Market,Wine Bar,Department Store,Chinese Restaurant
69,Libowu,Bus Station,Farmers Market,Wine Bar,Department Store,Chinese Restaurant
124,Wasimi Ojokoro,Bus Station,Farmers Market,Wine Bar,Department Store,Chinese Restaurant


In [49]:
#Cluster 7
lg_merged.loc[lg_merged['Cluster_Labels']==6, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
50,Iju,Ice Cream Shop,Wine Bar,Diner,Chinese Restaurant,Clothing Store
52,Iju Water Works,Ice Cream Shop,Wine Bar,Diner,Chinese Restaurant,Clothing Store


In [50]:
#Cluster 8
lg_merged.loc[lg_merged['Cluster_Labels']==7, lg_merged.columns[[0] + list(range(5, lg_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
110,Oworonshoki,Rental Car Location,Wine Bar,Department Store,Chinese Restaurant,Clothing Store
111,Oworonsoki,Rental Car Location,Wine Bar,Department Store,Chinese Restaurant,Clothing Store
